In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
store_100_path = r"df_all_store_100.parquet"
store_42_path = r"df_all_store_42.parquet"
store_4906_path = r"df_all_store_4906.parquet"

In [3]:
df_100 = pd.read_parquet(store_100_path)
df_42 = pd.read_parquet(store_42_path)
df_4906 = pd.read_parquet(store_4906_path)
df_all = pd.concat([df_100, df_42, df_4906])

In [4]:
del df_100, df_42, df_4906

In [5]:
df_all['date'] = pd.to_datetime(df_all['date'])

In [6]:
warehouse_path = r"df_all_store_VZ01.parquet"
df_warehouse = pd.read_parquet(warehouse_path)

In [7]:
df_warehouse = df_warehouse.rename(columns={"stock": "warehouse_stock"})
df_warehouse['date'] = pd.to_datetime(df_warehouse['date'])

In [8]:
df_all['sku'].nunique()

22823

In [9]:
df_warehouse['sku'] = df_warehouse['sku'].astype(str)


In [10]:
df_warehouse = df_warehouse[df_warehouse['date'] > '2018-01-01']

In [11]:
df_all = df_all.merge(df_warehouse[["sku","date","warehouse_stock"]], on = ["sku","date"], how = "left")


In [12]:
df_all = df_all[df_all['date'] > '2018-01-01']

In [13]:
df_all = df_all.sort_values('date')

In [14]:
df_all = df_all.merge(df_all[df_all['stock'] > 0].groupby(['sku', 'store']).agg({'stock': 'first'}).reset_index().rename(columns={'stock':'palmers_first_allocation'}), on = ['sku', 'store'], how = 'left')

In [15]:
sku_store_df_step_g = df_all.groupby(['sku', 'store'])

In [16]:
import json
dict_per_store_per_year_path = r"dict_per_store_per_year.json"
with open(dict_per_store_per_year_path) as json_file:
    dict_per_store_per_year = json.load(json_file)

In [17]:
first_dist_store_100 =  dict_per_store_per_year['2020']['100'] / (dict_per_store_per_year['2020']['100'] + dict_per_store_per_year['2020']['42'])
first_dist_store_42 =  dict_per_store_per_year['2020']['42'] / (dict_per_store_per_year['2020']['100'] + dict_per_store_per_year['2020']['42'])

In [18]:
first_dist_store_100, first_dist_store_42

(0.6374031057660117, 0.3625968942339883)

In [19]:
store_100_dist_need_to_be = 51 / 104
store_100_dist_need_to_be

0.49038461538461536

In [20]:
store_42_dist_need_to_be = 53 / 104
store_42_dist_need_to_be

0.5096153846153846

In [21]:
i = 0

dict_of_all_data_after_preprocess = {}
for sku in ['100630105000002']:
    dict_of_all_data_after_preprocess[sku] = {}
    for store in ['42', '100']:
        dict_of_all_data_after_preprocess[sku][store] = {}

        sku_store_df_step = sku_store_df_step_g.get_group((sku, store))
        first_stock = sku_store_df_step[sku_store_df_step["stock"]>0]["stock"].iloc[0]
        # nan in our_stock if before first stock
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "palmers_first_allocation"] = np.nan
        sku_store_df_step['days_from_first_stock'] = 1
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "days_from_first_stock"] = np.nan
        sku_store_df_step['days_from_first_stock'] = sku_store_df_step['days_from_first_stock'].cumsum()
        sku_store_df_step['sales_cumsum'] = sku_store_df_step['sales'].cumsum()
        sku_store_df_step['dynamic_stock_with_palmers_init'] = sku_store_df_step['palmers_first_allocation'] - sku_store_df_step['sales_cumsum']
        sku_store_df_step['sales_cumsum_divide_days_from_first_stock'] =  sku_store_df_step['sales_cumsum'] / sku_store_df_step['days_from_first_stock']

        zero_sales_days_back = 1
        sku_store_df_step['days_from_last_sales_that_not_zero'] = 1
        for index, row in sku_store_df_step.shift(1).iterrows():
            # if index is sku_store_df_step[sku_store_df_step["stock"]>0]["stock"] index
            if index > sku_store_df_step[sku_store_df_step["stock"]>0].index[0]:
                if row['sales'] == 0:
                    zero_sales_days_back += 1
                else:
                    zero_sales_days_back = 1
                sku_store_df_step.at[index, 'days_from_last_sales_that_not_zero'] = zero_sales_days_back
        sku_store_df_step.loc[sku_store_df_step["date"]<sku_store_df_step[sku_store_df_step["stock"]>0]["date"].iloc[0], "days_from_last_sales_that_not_zero"] = np.nan
        sku_store_df_step['rate_sales'] = sku_store_df_step['sales'] / sku_store_df_step['days_from_last_sales_that_not_zero']
        dict_of_all_data_after_preprocess[sku][store] = sku_store_df_step
        sku_store_df_step['diff_stock'] = sku_store_df_step['stock'].diff()




/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_43858/784705553.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sku_store_df_step['days_from_first_stock'] = 1
/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_43858/784705553.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sku_store_df_step['days_from_first_stock'] = sku_store_df_step['days_from_first_stock'].cumsum()
/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_43858/784705553.py:16: SettingWithCopyWarning: 
A val

In [22]:
print(sku_store_df_step.iloc[100:])

                      sku store       date  average_price  \
21180986  100630105000002   100 2020-12-11          24.99   
21205875  100630105000002   100 2020-12-12            NaN   
21229217  100630105000002   100 2020-12-13            NaN   
21278747  100630105000002   100 2020-12-14            NaN   
21307262  100630105000002   100 2020-12-15          24.99   
...                   ...   ...        ...            ...   
73205453  100630105000002   100 2023-12-08            NaN   
73248678  100630105000002   100 2023-12-09            NaN   
73300214  100630105000002   100 2023-12-10            NaN   
73355073  100630105000002   100 2023-12-11            NaN   
73388217  100630105000002   100 2023-12-12            NaN   

          average_dicounted_price  sales          item            sku_store  \
21180986                    24.99    1.0  100630105000  100630105000002,100   
21205875                      NaN    0.0  100630105000  100630105000002,100   
21229217                      

In [23]:

dynamic_distribution_per_store = {}
date = '2021-06-13'
total_stock_warehouse = 104
update_stock_remain = 104
warehouse_dict = {}
for sku in dict_of_all_data_after_preprocess.keys():
    dynamic_distribution_per_store[sku] = {}
    warehouse_dict[sku] = {}
    for store in dict_of_all_data_after_preprocess[sku].keys():
        dynamic_distribution_per_store[sku][store] = {}
        date_data = dict_of_all_data_after_preprocess[sku][store][dict_of_all_data_after_preprocess[sku][store]['date'] <= date]
        dynamic_distribution_per_store[sku][store]['initial_stock_in_store'] = date_data['palmers_first_allocation'].iloc[-1]
        dynamic_distribution_per_store[sku][store]['current_stock_in_store'] = date_data['dynamic_stock_with_palmers_init'].iloc[-1]
        dynamic_distribution_per_store[sku][store]['sales_cumsum'] =  date_data.set_index('date')['sales_cumsum']
        dynamic_distribution_per_store[sku][store]['rate_sales'] = date_data.set_index('date')['rate_sales']
        dynamic_distribution_per_store[sku][store]['sales_cumsum_divide_days_from_first_stock'] = date_data.set_index('date')['sales_cumsum_divide_days_from_first_stock']
        dynamic_distribution_per_store[sku][store]['initial_stock_in_warehouse'] = total_stock_warehouse
        update_stock_remain -= date_data['diff_stock'][date_data['diff_stock'] > 0].sum() - np.abs(date_data['sales'][date_data['sales'] < 0].sum())
        dynamic_distribution_per_store[sku][store]['prior_original_distribution'] = dict_per_store_per_year['2020'][store] / (dict_per_store_per_year['2020']['100'] + dict_per_store_per_year['2020']['42'])
        dynamic_distribution_per_store[sku][store]['prior_original_allocation_stock'] = np.round(dynamic_distribution_per_store[sku][store]['prior_original_distribution'] * total_stock_warehouse)
        dynamic_distribution_per_store[sku][store]['sales'] = date_data.set_index('date')['sales']




    warehouse_dict[sku]['initial_stock_in_warehouse'] = total_stock_warehouse
    warehouse_dict[sku]['remain_stock'] = update_stock_remain
    for store in dict_of_all_data_after_preprocess[sku].keys():
        dynamic_distribution_per_store[sku][store]['initial_stock_in_warehouse'] = warehouse_dict[sku]['initial_stock_in_warehouse']
        dynamic_distribution_per_store[sku][store]['remain_stock'] = warehouse_dict[sku]['remain_stock']



In [24]:
dynamic_distribution_per_store

{'100630105000002': {'42': {'initial_stock_in_store': 20.0,
   'current_stock_in_store': -24.0,
   'sales_cumsum': date
   2020-09-02     0.0
   2020-09-03     0.0
   2020-09-04     0.0
   2020-09-05     0.0
   2020-09-06     0.0
                 ... 
   2021-06-09    44.0
   2021-06-10    44.0
   2021-06-11    44.0
   2021-06-12    44.0
   2021-06-13    44.0
   Name: sales_cumsum, Length: 285, dtype: float64,
   'rate_sales': date
   2020-09-02    NaN
   2020-09-03    NaN
   2020-09-04    NaN
   2020-09-05    NaN
   2020-09-06    NaN
                ... 
   2021-06-09    0.0
   2021-06-10    0.0
   2021-06-11    0.0
   2021-06-12    0.0
   2021-06-13    0.0
   Name: rate_sales, Length: 285, dtype: float64,
   'sales_cumsum_divide_days_from_first_stock': date
   2020-09-02         NaN
   2020-09-03         NaN
   2020-09-04         NaN
   2020-09-05         NaN
   2020-09-06         NaN
                   ...   
   2021-06-09    0.161765
   2021-06-10    0.161172
   2021-06-11    0.160

In [25]:
import warnings
warnings.filterwarnings("ignore")
for sku in dynamic_distribution_per_store:
    df_of_sales_cumsum_divide_days_from_first_stock = pd.DataFrame()
    df_of_sales_rolling = pd.DataFrame()
    df_of_dynamic_distribution_per_store = pd.DataFrame()
    df_of_sales_rolling_factor = pd.DataFrame()
    for store in dynamic_distribution_per_store[sku]:
        initial_dist = dynamic_distribution_per_store[sku][store]['prior_original_distribution']
        dynamic_dist = pd.Series([np.nan] * len(dynamic_distribution_per_store[sku][store]['sales_cumsum']),
                                 index=dynamic_distribution_per_store[sku][store]['sales_cumsum'].index)
        first_index_that_no_nan = dynamic_distribution_per_store[sku][store]['sales_cumsum_divide_days_from_first_stock'].first_valid_index()

        dynamic_dist.loc[first_index_that_no_nan] = initial_dist
        # delete dynamic_dist rows after first_index_that_no_nan
        dynamic_dist = dynamic_dist.loc[:first_index_that_no_nan]

        dynamic_distribution_per_store[sku][store]['dynamic_dist'] = dynamic_dist


        df_of_sales_cumsum_divide_days_from_first_stock[store] = dynamic_distribution_per_store[sku][store]['sales_cumsum_divide_days_from_first_stock']

        df_of_sales_rolling[f'{store}'] = dynamic_distribution_per_store[sku][store]['sales'].rolling(7).sum()
        df_of_dynamic_distribution_per_store[store] = dynamic_distribution_per_store[sku][store]['dynamic_dist']
   # df_of_dynamic_distribution_per_store = df_of_dynamic_distribution_per_store.ffill()
    A1 =  df_of_sales_rolling.sub(df_of_sales_rolling.min(axis=1), axis=0)
    A2 = A1.divide(A1.max(axis=1), axis=0).fillna(0)

    df_of_sales_rolling_factor = A2



In [53]:
k = 0
for index, row in df_of_sales_cumsum_divide_days_from_first_stock.iterrows():
    print(index)
    if row.isnull().all():
        pass
    else:
        k += 1
        if k == 1:
            pass

        else:
            dynamic_distribution_per_store_row = df_of_dynamic_distribution_per_store.loc[index - pd.Timedelta(days=1)]
            df_of_sales_rolling_factor_row = df_of_sales_rolling_factor.loc[index]
            if sum(df_of_sales_rolling_factor_row) == 0:
                # replace in uniuon disterbution. if there is 3 stores and all of them have 0 sales, then we will replace with 1/3
                df_of_sales_rolling_factor_row = [1 / len(dynamic_distribution_per_store_row)] * len(dynamic_distribution_per_store_row)
            new_dist_list = []
            for i in range(len(dynamic_distribution_per_store_row)):
                old_dist = dynamic_distribution_per_store_row.iloc[i]
                new_dist = old_dist + df_of_sales_rolling_factor_row[i] * 0.01
                new_dist_list.append(new_dist)

            new_dist_list_norm = [x / sum(new_dist_list) for x in new_dist_list]
            new_dist_list_norm = pd.DataFrame([new_dist_list_norm], columns=df_of_dynamic_distribution_per_store.columns, index=[index])
            print("######")
            print(new_dist_list_norm)
            df_of_dynamic_distribution_per_store = df_of_dynamic_distribution_per_store.append(new_dist_list_norm)


2020-09-02 00:00:00
2020-09-03 00:00:00
2020-09-04 00:00:00
2020-09-05 00:00:00
2020-09-06 00:00:00
2020-09-07 00:00:00
2020-09-08 00:00:00
2020-09-09 00:00:00
2020-09-10 00:00:00
2020-09-11 00:00:00
2020-09-12 00:00:00
######
                  42       100
2020-09-12  0.359007  0.640993
2020-09-13 00:00:00
######
                                                           42  \
2020-09-13  42     0.495369
100    0.494868
Name: 2020-09-...   

                                                          100  
2020-09-13  42     0.504631
100    0.505132
Name: 2020-09-...  
2020-09-14 00:00:00
######
                                                           42  \
2020-09-14  42     42     0.417268
100    0.417510
Name: 2...   

                                                          100  
2020-09-14  42     42     0.582732
100    0.582490
Name: 2...  
2020-09-15 00:00:00
######
                                                           42  \
2020-09-15  42     42     42     0.464656
100  

/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_43858/665493372.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_of_dynamic_distribution_per_store = df_of_dynamic_distribution_per_store.append(new_dist_list_norm)
/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_43858/665493372.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_of_dynamic_distribution_per_store = df_of_dynamic_distribution_per_store.append(new_dist_list_norm)
/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_43858/665493372.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_of_dynamic_distribution_per_store = df_of_dynamic_distribution_per_store.append(new_dist_list_norm)
/var/folders/sv/p02my_xx7t193mwjyb2ytt04

                                                           42  \
2020-09-21  42     42     42     42     42     42     42  ...   

                                                          100  
2020-09-21  42     42     42     42     42     42     42  ...  
2020-09-22 00:00:00
######


/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_43858/665493372.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_of_dynamic_distribution_per_store = df_of_dynamic_distribution_per_store.append(new_dist_list_norm)


                                                           42  \
2020-09-22  42     42     42     42     42     42     42  ...   

                                                          100  
2020-09-22  42     42     42     42     42     42     42  ...  
2020-09-23 00:00:00


...

######
                                                           42  \
2020-09-27  42     42     42     42     42     42     42  ...   

                                                          100  
2020-09-27  42     42     42     42     42     42     42  ...  
2020-09-28 00:00:00


/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_43858/665493372.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_of_dynamic_distribution_per_store = df_of_dynamic_distribution_per_store.append(new_dist_list_norm)


######
                                                           42  \
2020-09-28  42     42     42     42     42     42     42  ...   

                                                          100  
2020-09-28  42     42     42     42     42     42     42  ...  
2020-09-29 00:00:00


/var/folders/sv/p02my_xx7t193mwjyb2ytt040000gn/T/ipykernel_43858/665493372.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_of_dynamic_distribution_per_store = df_of_dynamic_distribution_per_store.append(new_dist_list_norm)


KeyboardInterrupt: 

In [52]:
df_of_dynamic_distribution_per_store.iloc[8:30]

,42,100
2020-09-10,NaN,NaN
2020-09-11,0.362597,0.637403
2020-09-12,0.362235,0.637765
2020-09-13,0.361873,0.638127
2020-09-14,0.361511,0.638489
2020-09-15,0.361650,0.638350
2020-09-16,0.361788,0.638212
2020-09-17,0.361426,0.638574
2020-09-18,0.361065,0.638935
2020-09-19,0.361204,0.638796


In [28]:
df_of_dynamic_distribution_per_store.iloc[:10]

,42,100
2020-09-02,NaN,NaN
2020-09-03,NaN,NaN
2020-09-04,NaN,NaN
2020-09-05,NaN,NaN
2020-09-06,NaN,NaN
2020-09-07,NaN,NaN
2020-09-08,NaN,NaN
2020-09-09,NaN,NaN
2020-09-10,NaN,NaN
2020-09-11,0.362597,0.637403


In [29]:
df_of_sales_rolling_factor_row[i]

0.5

In [30]:
old_dist

0.6022184797757001

In [31]:
new_dist

0.6027184797757

In [32]:
df_of_sales_rolling_factor_row[i]

0.5

In [33]:
new_dist_list

[0.3982815202243001, 0.6027184797757]

In [34]:
df_of_dynamic_distribution_per_store

,42,100
2020-09-02,NaN,NaN
2020-09-03,NaN,NaN
2020-09-04,NaN,NaN
2020-09-05,NaN,NaN
2020-09-06,NaN,NaN
...,...,...
2021-06-09,0.397475,0.602525
2021-06-10,0.397577,0.602423
2021-06-11,0.397679,0.602321
2021-06-12,0.397782,0.602218


In [35]:
new_dist_list_norm

,42,100
2021-06-13,0.397884,0.602116


In [36]:
df_of_dynamic_distribution_per_store

,42,100
2020-09-02,NaN,NaN
2020-09-03,NaN,NaN
2020-09-04,NaN,NaN
2020-09-05,NaN,NaN
2020-09-06,NaN,NaN
...,...,...
2021-06-09,0.397475,0.602525
2021-06-10,0.397577,0.602423
2021-06-11,0.397679,0.602321
2021-06-12,0.397782,0.602218


In [37]:
new_df = pd.DataFrame([new_dist_list], columns=df_of_dynamic_distribution_per_store.columns, index=[index])
new_df

,42,100
2021-06-13,0.398282,0.602718


In [38]:
df_of_dynamic_distribution_per_store.index

DatetimeIndex(['2020-09-02', '2020-09-03', '2020-09-04', '2020-09-05',
               '2020-09-06', '2020-09-07', '2020-09-08', '2020-09-09',
               '2020-09-10', '2020-09-11',
               ...
               '2021-06-04', '2021-06-05', '2021-06-06', '2021-06-07',
               '2021-06-08', '2021-06-09', '2021-06-10', '2021-06-11',
               '2021-06-12', '2021-06-13'],
              dtype='datetime64[ns]', length=285, freq=None)

In [39]:
new_dist_list_norm

,42,100
2021-06-13,0.397884,0.602116


In [40]:
pd.DataFrame(new_dist_list_norm, index=dynamic_distribution_per_store_row.index, columns=df_of_dynamic_distribution_per_store.columns)

,42,100
42,NaN,NaN
100,NaN,NaN


In [41]:
new_dist_list_norm

,42,100
2021-06-13,0.397884,0.602116


In [42]:
df_of_dynamic_distribution_per_store

,42,100
2020-09-02,NaN,NaN
2020-09-03,NaN,NaN
2020-09-04,NaN,NaN
2020-09-05,NaN,NaN
2020-09-06,NaN,NaN
...,...,...
2021-06-09,0.397475,0.602525
2021-06-10,0.397577,0.602423
2021-06-11,0.397679,0.602321
2021-06-12,0.397782,0.602218


In [43]:
old_dist

0.6022184797757001

In [44]:
new_dist_list_norm

,42,100
2021-06-13,0.397884,0.602116


In [45]:
df_of_sales_cumsum_divide_days_from_first_stock

,42,100
date,,
2020-09-02,NaN,NaN
2020-09-03,NaN,NaN
2020-09-04,NaN,NaN
2020-09-05,NaN,NaN
2020-09-06,NaN,NaN
...,...,...
2021-06-09,0.161765,0.143382
2021-06-10,0.161172,0.142857
2021-06-11,0.160584,0.142336


In [46]:
dynamic_distribution_per_store_row

42     0.397782
100    0.602218
Name: 2021-06-12 00:00:00, dtype: float64

In [47]:
dynamic_distribution_per_store

{'100630105000002': {'42': {'initial_stock_in_store': 20.0,
   'current_stock_in_store': -24.0,
   'sales_cumsum': date
   2020-09-02     0.0
   2020-09-03     0.0
   2020-09-04     0.0
   2020-09-05     0.0
   2020-09-06     0.0
                 ... 
   2021-06-09    44.0
   2021-06-10    44.0
   2021-06-11    44.0
   2021-06-12    44.0
   2021-06-13    44.0
   Name: sales_cumsum, Length: 285, dtype: float64,
   'rate_sales': date
   2020-09-02    NaN
   2020-09-03    NaN
   2020-09-04    NaN
   2020-09-05    NaN
   2020-09-06    NaN
                ... 
   2021-06-09    0.0
   2021-06-10    0.0
   2021-06-11    0.0
   2021-06-12    0.0
   2021-06-13    0.0
   Name: rate_sales, Length: 285, dtype: float64,
   'sales_cumsum_divide_days_from_first_stock': date
   2020-09-02         NaN
   2020-09-03         NaN
   2020-09-04         NaN
   2020-09-05         NaN
   2020-09-06         NaN
                   ...   
   2021-06-09    0.161765
   2021-06-10    0.161172
   2021-06-11    0.160

In [48]:
print(df_of_sales_rolling_factor)

             42  100
date                
2020-09-02  0.0  0.0
2020-09-03  0.0  0.0
2020-09-04  0.0  0.0
2020-09-05  0.0  0.0
2020-09-06  0.0  0.0
...         ...  ...
2021-06-09  0.0  0.0
2021-06-10  0.0  0.0
2021-06-11  0.0  0.0
2021-06-12  0.0  0.0
2021-06-13  0.0  0.0

[285 rows x 2 columns]
